

# Attorney Finder

### Imports

### Using Other Attorneys

### Using a Brief

In [1]:
from IPython.display import Javascript, display, HTML
from ipywidgets import widgets

def go_next(ev):
    display(Javascript('IPython.notebook.execute_cell_range(Jupyter.notebook.find_cell_index(Jupyter.notebook.get_selected_cell())+1,Jupyter.notebook.find_cell_index(Jupyter.notebook.get_selected_cell())+2)'))
    Javascript('IPython.notebook.move_selection_down')
    
def go_brief(ev):
    display(Javascript('IPython.notebook.execute_cell_range(Jupyter.notebook.find_cell_index(Jupyter.notebook.get_selected_cell())+2,Jupyter.notebook.find_cell_index(Jupyter.notebook.get_selected_cell())+3)'))
    Javascript('IPython.notebook.move_selection_down')
    Javascript('IPython.notebook.move_selection_down')
    
from fuzzywuzzy import process
from operator import itemgetter
from IPython.display import clear_output

import pandas as pd
import numpy as np
import json
import requests
from googleapiclient.discovery import build



import itertools
from scipy.sparse import csr_matrix, coo_matrix

import warnings
warnings.filterwarnings('ignore')

full = pd.read_csv('./data/atty_finder.csv')
full = full.drop('Unnamed: 0', axis=1)
pt_toggle = widgets.Checkbox(value=False, description='Poly-topic weighting?')
display(pt_toggle)

input_type = widgets.Dropdown(options=['document', 'attorney'], value='document', description='Method?:')
display(input_type)

next_button = widgets.Button(description='Continue')
display(next_button)
next_button.on_click(go_next)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [2]:
del M

NameError: name 'M' is not defined

In [4]:
pt_weighting = pt_toggle.value
option = input_type.value

try:
    del A
except:
    pass
try:
    del M
except:
    pass

try:
    emails = list(np.unique(full['email']))
    firms = list(np.unique(np.array([i.split('@')[1].split('.')[0] for i in emails])))
    attorney_map = {}
    case_map = {}
    attorney_idx = 0
    case_idx = 0

    for index,email,case_id, weight,updated_NOS, cites_common_count, pt_score in full.itertuples():
        if email not in attorney_map:
            attorney_map[email] = attorney_idx
            attorney_idx+=1
        if case_id not in case_map:
            case_map[case_id] = case_idx
            case_idx+=1
    del full
except:
    pass
    
if option == 'document':
    if pt_weighting:
        A = np.load('./data/A_pt_weight.npy')
        go_brief(True)
    else:
        A = np.load('./data/A_no_pt_weight.npy')
        go_brief(True)
else:
    if pt_weighting:
        M = np.load('./data/M_pt_weight.npy')
        go_next(True)
    else:
        M = np.load('./data/M_no_pt_weight.npy')
        go_next(True)

        

# I,J, data = [],[],[]
# for index, attorney, case_id, weight,updated_NOS, cites_common_count, pt_score in full.itertuples():
#     I.append(attorney_map[attorney])
#     J.append(case_map[case_id])
#     if pt_weighting:
#         data.append(weight * pt_score)
#     else:
#         data.append(weight)
# n = len(attorney_map)
# m = len(case_map)
# M = coo_matrix((data, (I, J)), (n, m))
# A = M.tocsr()
# M = (A.dot(A.transpose())) 
# for i in range(M.shape[0]):
#     M[i,i] = 0
# del I,J,data

<IPython.core.display.Javascript object>

In [5]:
def top_results(attorneys, matrix1=M, num_results=5, end1=emails, begin1=attorney_map, printing1 = True):
    A = matrix1[()].shape[0]
    I, J, V = [], [], []
    for i in attorneys:
        I.append(attorney_map[i])
        J.append(0)
        V.append(1)
    q_vec = coo_matrix((V,(I,J)),shape=(A, 1))
    resultant_vec = matrix1.dot(q_vec).todense()
    results = [item for sublist in resultant_vec.tolist() for item in sublist]
    results_indexed = list(enumerate(results))
    sorted_results = sorted(results_indexed, key=itemgetter(1))
    final_index = sorted_results[-num_results:]
    return final_index, resultant_vec

def two_pass(attorneys, matrix=M, first_pass =20, second_pass = 5, end=emails, begin=attorney_map, printing = False, print_out = True):
    index, vector = top_results(attorneys, matrix1=matrix, num_results=first_pass, end1=end, begin1=begin, printing1=printing)
    vector = np.matrix(np.array([0 for i in range(M[()].shape[0])]).reshape(M[()].shape[0], 1))
    for i in index:
        vector = vector + top_results([emails[i[0]]])[1]
    for i in attorneys:
        vector[attorney_map[i]] = 0
    results = [item for sublist in vector.tolist() for item in sublist]
    results_indexed = list(enumerate(results))
    sorted_results = sorted(results_indexed, key=itemgetter(1))
    final_index = sorted_results[-second_pass:]
    attorneys_res = []
    for i in final_index:
        attorneys_res.append(end[i[0]])
    if print_out:
        count = second_pass
        for i in attorneys_res:
            display(HTML('Attorney number %d: <a href="https://www.google.com/search?q=%s", target="_blank">%s</a>' %(count, str(i), str(i))))
            count -=1
    return attorneys_res

def atty_choose(firms):
    atty_show = [i for i in emails if i.split('@')[1].split('.')[0] in firms]
    atty_choices = widgets.SelectMultiple(options=atty_show,description='Attorneys')
    display(atty_choices)

    one_pass = widgets.IntSlider(min=0,max=20,step=1,value=5, description='1st pass:')
    display(one_pass)

    sec_pass = widgets.IntSlider(min=0,max=20,step=1,value=5, description='2nd pass:')
    display(sec_pass)

#     print_first = widgets.Checkbox(value=False, description='See primary attorneys?',)
#     display(print_first)

    def button(b):
        two_pass(list(atty_choices.value), 
                 matrix=M, 
                 first_pass = one_pass.value, 
                 second_pass = sec_pass.value, 
                 end=emails, 
                 begin=attorney_map, 
                 printing = False, 
                 print_out = True)
        
    run = widgets.Button(description="Click To Run!")
    display(run)
    run.on_click(button)
    
def firm_confirm(text):
    firms_show = [i[0] for i in process.extract(text.value, firms, limit=3) if i[1] > 85]
    firm_choices = widgets.SelectMultiple(options=firms_show,description='Firms')
    display(firm_choices)
    def button(b):
        atty_choose(firm_choices.value)
    firm_show_button = widgets.Button(description="Submit!")
    display(firm_show_button)
    firm_show_button.on_click(button)
    
def firm_choose(b):
    start.close()
    firm_text = widgets.Text(description='Firm:')
    display(firm_text)
    def button(b):
        firm_confirm(firm_text)
    firm_button = widgets.Button(description="Submit")
    display(firm_button)
    firm_button.on_click(button)
    
button = widgets.Button(description="Click To Reset!")
display(button)

def reset(ev):
    display(Javascript('IPython.notebook.execute_cell()'))

button.on_click(reset)
start = widgets.Button(description='Click to Begin!')
display(start)
start.on_click(firm_choose)

In [3]:
from os import listdir
from os.path import isfile, join


def get_referral(choice):
    from extract_references_from_pdfs import process_file
    choice = choice
    refs = process_file('./docs/'+choice)
    refs = refs.json()['references']
    I,J, data = [],[],[]
    for i in refs:
        try:
            I.append(case_map[i['id']])
            J.append(0)
            data.append(i['weight']) 
        except KeyError:
            continue
    n = len(case_map)
    m = 1
    X = coo_matrix((data, (I, J)), (n, m))
    del I, J, data

    num_results = 5
    resultant_vec = A.dot(X).todense()
    results = [item for sublist in resultant_vec.tolist() for item in sublist]
    results_indexed = list(enumerate(results))
    sorted_results = sorted(results_indexed, key=itemgetter(1))
    final_index = sorted_results[-num_results:]
    attorneys_res = []
    for i in final_index:
            attorneys_res.append(emails[i[0]])
    for i in attorneys_res:
        display(HTML('Attorney number %d: <a href="https://www.google.com/search?q=%s", target="_blank">%s</a>' %(num_results, str(i), str(i))))
        num_results -=1

def doc_choose(ev):
    onlyfiles = [f for f in listdir('./docs/') if isfile(join('./docs/',f))]
    doc_choice = widgets.Dropdown(options=onlyfiles, description='PDF Document?:')
    display(doc_choice)
    next_button = widgets.Button(description='Continue')
    display(next_button)
    def button(b):
        get_referral(doc_choice.value)
    next_button.on_click(button)


button = widgets.Button(description="Click To Reset!")
display(button)

def reset(ev):
    display(Javascript('IPython.notebook.execute_cell()'))

button.on_click(reset)
start = widgets.Button(description='Click to Begin!')
display(start)
start.on_click(doc_choose)